In [59]:
# BÀI 1: Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức 
import pandas as pd
import numpy as np
import sqlite3
import math
# Tạo DataFrame với giá trị ngẫu nhiên từ 1 đến 20, gồm 10 hàng
df = pd.DataFrame({
    'A': np.random.randint(1, 21, size=10),  
    'B': np.random.randint(1, 21, size=10)
})
# Tạo kết nối SQLite 
conn = sqlite3.connect("chuong3")
df.to_sql("bai1", conn, index=False, if_exists="replace")

# Truy vấn các giá trị cần tính
query = """
SELECT
    COUNT(*) as n,
    SUM(A) as sum_a,
    SUM(B) as sum_b,
    SUM(A * B) as sum_ab,
    SUM(A * A) as sum_a2,
    SUM(B * B) as sum_b2
FROM bai1;
"""

row = conn.execute(query).fetchone()
n, sum_a, sum_b, sum_ab, sum_a2, sum_b2 = row

# Áp dụng công thức Pearson
numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt(n * sum_a2 - sum_a**2) * math.sqrt(n * sum_b2 - sum_b**2)
r = numerator / denominator

print(f"Hệ số tương quan Pearson r = {r:4f}")

Hệ số tương quan Pearson r = -0.217024


In [60]:
# BÀI 2 Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày
#  và chấm điểm theo thang từ 1đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không?
#  Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2
import pandas as pd
from scipy.stats import chi2_contingency

#1 Tạo DataFrame từ bảng
data = pd.DataFrame({
    'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4'],
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6],
    'C': [7, 7, 8, 5]
})
# Kết nối SQLite 
conn = sqlite3.connect('chuong3')

# Đẩy dữ liệu vào SQLite
data.to_sql("bai2", conn, index=False, if_exists="replace")

# Kiểm tra xem có dữ liệu chưa
print(pd.read_sql("SELECT * FROM bai2 LIMIT 5;", conn))
query_long = """
SELECT Day, 'A' AS Model, A AS Score FROM bai2
UNION ALL
SELECT Day, 'B' AS Model, B AS Score FROM bai2
UNION ALL
SELECT Day, 'C' AS Model, C AS Score FROM bai2
"""
data_long = pd.read_sql(query_long, conn)


     Day    A    B  C
0  Day 1  8.0  9.0  7
1  Day 2  7.5  8.5  7
2  Day 3  6.0  7.0  8
3  Day 4  7.0  6.0  5


In [61]:
# 2 Tạo bảng chéo để kiểm định chi-squared
# Làm tròn điểm để phân loại 
data_long['Score_rounded'] = data_long['Score'].round().astype(int)

# Bảng chéo theo mẫu xe
table_model = pd.crosstab(data_long['Model'], data_long['Score_rounded'])

# Thực hiện kiểm định chi-squared
chi2, p, _, _ = chi2_contingency(table_model)
# In kết quả
print(f" Kiểm định theo mẫu xe: p-value = {p:.4f}")
print("\nBảng chéo (Contingency Table):")
print(table_model)
print("\nKết quả kiểm định chi-squared:")
print(f"Chi-squared statistic = {chi2:.4f}")
print(f"P-value = {p:.4f}")
# Đánh giá
if p >= 0.05:
    print("Bác bỏ ho => Không có sự khác biệt đáng kể giữa các mẫu xe")
else:
    print("Chấp nhận ho => Có sự khác biệt đáng kể giữa các mẫu xe dựa trên điểm số")

 Kiểm định theo mẫu xe: p-value = 0.6472

Bảng chéo (Contingency Table):
Score_rounded  5  6  7  8  9
Model                       
A              0  1  1  2  0
B              0  1  1  1  1
C              1  0  2  1  0

Kết quả kiểm định chi-squared:
Chi-squared statistic = 6.0000
P-value = 0.6472
Bác bỏ ho => Không có sự khác biệt đáng kể giữa các mẫu xe


In [62]:
import sqlite3
import pandas as pd

# Tạo kết nối đến SQLite DB
conn = sqlite3.connect('chuong3')

# Xóa bảng nếu đã tồn tại (nên đặt trước khi tạo)
conn.execute("DROP TABLE IF EXISTS flights")

# Tạo bảng flights
conn.execute("CREATE TABLE flights (departure_time INTEGER)")

# Thêm dữ liệu mẫu
conn.executemany("INSERT INTO flights (departure_time) VALUES (?)", [
    (30,), (830,), (945,), (1445,), (2359,)
])

# Truy vấn để định dạng lại thời gian
query = """
SELECT 
  departure_time,
  printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights;
"""

# Đọc kết quả bằng pandas
df = pd.read_sql(query, conn)
print(df)


   departure_time formatted_time
0              30          00:30
1             830          08:30
2             945          09:45
3            1445          14:45
4            2359          23:59


In [64]:
# BÀI 4 Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa
import pandas as pd
import numpy as np
import sqlite3
# Ví dụ DataFrame
data = pd.DataFrame({'value': [4,11,5,6,8,7,9,80,64]})  # có vài giá trị ngoại lệ


# Tạo DB SQLite  và thêm dữ liệu
conn = sqlite3.connect('chuong3')
conn.execute("DROP TABLE IF EXISTS bai4")
data.to_sql('bai4', conn, index=False, if_exists='replace')

# Lấy dữ liệu từ SQL để tính median & MAD trong Python
df = pd.read_sql("SELECT value FROM bai4", conn)

# Bước 1: Tính median
median = df['value'].median()

# Bước 2: Tính MAD
mad = (df['value'] - median).abs().median()

# Bước 3: Tìm giá trị ngoại lệ (dùng lại SQLite để lọc)
threshold = 1.5 * mad
# Truy vấn lấy tất cả dữ liệu kèm cột is_outlier = True/False
query_flag = f"""
SELECT 
  value,
  CASE 
    WHEN ABS(value - {median}) > {threshold} THEN 1
    ELSE 0
  END AS is_outlier
FROM bai4
"""
result = pd.read_sql(query_flag, conn)
result['is_outlier'] = result['is_outlier'].astype(bool)

print("-- Kết quả có cờ is_outlier --")
print(result)

conn.close()




-- Kết quả có cờ is_outlier --
   value  is_outlier
0      4       False
1     11       False
2      5       False
3      6       False
4      8       False
5      7       False
6      9       False
7     80        True
8     64        True


In [ ]:
# BÀI 5 Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”
import sqlite3
import pandas as pd

# Kết nối SQLite 
conn = sqlite3.connect('chuong3')
conn.execute('DROP TABLE IF EXISTS Patient')

# Tạo bảng Patient
conn.execute('''
CREATE TABLE Patient (
    last_name TEXT,
    weight REAL,
    height REAL
)
''')

# Thêm dữ liệu mẫu
patients = [
    ('Le', 50, 160),
    ('Nguyen', 65, 171),
    ('Tran', 60, 168),
    ('Nguyen', 70, 172),
    ('Mai', 59.9, 168),
    ('Le', 50, 160)
]
conn.executemany('INSERT INTO Patient VALUES (?, ?, ?)', patients)

# Truy vấn tìm các cặp có thể là cùng một người
query = '''
SELECT 
    A.rowid AS id1,
    B.rowid AS id2
FROM Patient A
JOIN Patient B
    ON A.rowid < B.rowid
WHERE A.last_name = B.last_name AND ABS(A.weight - B.weight) <= 1
'''

# Đọc kết quả
matches = conn.execute(query).fetchall()

# In kết quả theo định dạng yêu cầu
print("Các cặp được cho là cùng một người (dựa trên last_name và weight):")
for match in matches:
    print(f"- Patient {match[0]} và Patient {match[1]}")

Các cặp được cho là cùng một người (dựa trên last_name và weight):
- Patient 1 và Patient 6
